## Computational Theory Assessment

In [ ]:
import numpy as np
import os

# Hide overflow warnings
np.seterr(over='ignore')

{'divide': 'warn', 'over': 'ignore', 'under': 'ignore', 'invalid': 'warn'}

## Problem 1: Binary Words and Operations

In [35]:
def Parity(x, y, z):
    """Calculate parity using XOR: x ⊕ y ⊕ z"""
    return np.uint32(x ^ y ^ z)

def Ch(x, y, z):
    """Choose function: (x AND y) XOR (NOT x AND z)"""
    return np.uint32((x & y) ^ (~x & z))

def Maj(x, y, z):
    """Majority function: (x AND y) XOR (x AND z) XOR (y AND z)"""
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

def ROTR(x, n):
    """Rotate right by n positions"""
    return np.uint32((x >> n) | (x << (32 - n)))

def SHR(x, n):
    """Shift right by n positions"""
    return np.uint32(x >> n)

def Sigma0(x):
    """Σ₀ function: ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)"""
    return np.uint32(ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22))

def Sigma1(x):
    """Σ₁ function: ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)"""
    return np.uint32(ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25))

def sigma0(x):
    """σ₀ function: ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)"""
    return np.uint32(ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3))

def sigma1(x):
    """σ₁ function: ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)"""
    return np.uint32(ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10))

In [36]:
# Test Problem 1 functions
x = np.uint32(0xAAAAAAAA)
y = np.uint32(0x55555555)
z = np.uint32(0xFFFFFFFF)

print("Testing binary operations:")
print(f"Parity: {Parity(x, y, z):#010x}")
print(f"Ch: {Ch(x, y, z):#010x}")
print(f"Maj: {Maj(x, y, z):#010x}")
print(f"Sigma0: {Sigma0(x):#010x}")
print(f"Sigma1: {Sigma1(x):#010x}")
print(f"sigma0: {sigma0(x):#010x}")
print(f"sigma1: {sigma1(x):#010x}")

Testing binary operations:
Parity: 0x00000000
Ch: 0x55555555
Maj: 0xffffffff
Sigma0: 0x55555555
Sigma1: 0xaaaaaaaa
sigma0: 0xeaaaaaaa
sigma1: 0x002aaaaa


## Problem 2: Fractional Parts of Cube Roots

In [37]:
def primes(n):
    """Generate first n prime numbers using Sieve of Eratosthenes"""
    if n <= 0:
        return []
    
    if n < 6:
        limit = 15
    else:
        import math
        limit = int(n * (math.log(n) + math.log(math.log(n)))) + 100
    
    sieve = [True] * limit
    sieve[0] = sieve[1] = False
    
    for i in range(2, int(limit**0.5) + 1):
        if sieve[i]:
            for j in range(i*i, limit, i):
                sieve[j] = False
    
    result = []
    for i in range(limit):
        if sieve[i]:
            result.append(i)
            if len(result) == n:
                break
    
    return result

In [38]:
# Calculate K constants
first_64_primes = primes(64)
K_constants = []

for prime in first_64_primes:
    cube_root = prime ** (1/3)
    fractional_part = cube_root - int(cube_root)
    first_32_bits = int(fractional_part * (2**32))
    K_constants.append(np.uint32(first_32_bits))

print(f"Calculated {len(K_constants)} constants")
print(f"First 8: {[f'{k:#010x}' for k in K_constants[:8]]}")

Calculated 64 constants
First 8: ['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5']


In [39]:
# Verify against standard
STANDARD_K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

STANDARD_K = [np.uint32(k) for k in STANDARD_K]

all_match = all(calc == std for calc, std in zip(K_constants, STANDARD_K))
print(f"Constants match standard: {all_match}")

Constants match standard: True


## Problem 3: Padding

In [40]:
def block_parse(msg):
    """Pad message and yield 512-bit blocks"""
    msg_len_bits = len(msg) * 8
    padded = bytearray(msg)
    padded.append(0x80)
    
    while len(padded) % 64 != 56:
        padded.append(0x00)
    
    padded.extend(msg_len_bits.to_bytes(8, byteorder='big'))
    
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])

In [41]:
# Test padding
test_messages = [b"", b"abc", b"a" * 55, b"a" * 56]

for msg in test_messages:
    blocks = list(block_parse(msg))
    print(f"Message length: {len(msg):3d} bytes -> {len(blocks)} block(s) of 64 bytes each")

Message length:   0 bytes -> 1 block(s) of 64 bytes each
Message length:   3 bytes -> 1 block(s) of 64 bytes each
Message length:  55 bytes -> 1 block(s) of 64 bytes each
Message length:  56 bytes -> 2 block(s) of 64 bytes each


## Problem 4: Hashes

In [42]:
# Initial hash values
INITIAL_HASH = [
    np.uint32(0x6a09e667),
    np.uint32(0xbb67ae85),
    np.uint32(0x3c6ef372),
    np.uint32(0xa54ff53a),
    np.uint32(0x510e527f),
    np.uint32(0x9b05688c),
    np.uint32(0x1f83d9ab),
    np.uint32(0x5be0cd19)
]

In [43]:
def hash(current, block):
    """SHA-256 compression function"""
    H = [np.uint32(h) for h in current]
    W = []
    
    # Prepare message schedule
    for i in range(16):
        word = int.from_bytes(block[i*4:(i+1)*4], byteorder='big')
        W.append(np.uint32(word))
    
    for t in range(16, 64):
        w = np.uint32(sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16])
        W.append(w)
    
    # Initialize working variables
    a, b, c, d, e, f, g, h = H
    
    # Main loop
    for t in range(64):
        T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + STANDARD_K[t] + W[t])
        T2 = np.uint32(Sigma0(a) + Maj(a, b, c))
        h = g
        g = f
        f = e
        e = np.uint32(d + T1)
        d = c
        c = b
        b = a
        a = np.uint32(T1 + T2)
    
    # Add to current hash
    H[0] = np.uint32(H[0] + a)
    H[1] = np.uint32(H[1] + b)
    H[2] = np.uint32(H[2] + c)
    H[3] = np.uint32(H[3] + d)
    H[4] = np.uint32(H[4] + e)
    H[5] = np.uint32(H[5] + f)
    H[6] = np.uint32(H[6] + g)
    H[7] = np.uint32(H[7] + h)
    
    return H

In [44]:
def sha256(message):
    """Complete SHA-256 hash function"""
    current_hash = INITIAL_HASH.copy()
    
    for block in block_parse(message):
        current_hash = hash(current_hash, block)
    
    result = ''.join(f'{h:08x}' for h in current_hash)
    return result

In [45]:
# Test SHA-256 implementation
test_vectors = [
    (b"", "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"),
    (b"abc", "ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad"),
    (b"abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq",
     "248d6a61d20638b8e5c026930c3e6039a33ce45964ff2167f6ecedd419db06c1")
]

print("Testing SHA-256:")
for msg, expected in test_vectors:
    result = sha256(msg)
    match = result == expected
    print(f"{str(msg)[:30]:30s} -> Match: {match}")

Testing SHA-256:
b''                            -> Match: True
b'abc'                         -> Match: True
b'abcdbcdecdefdefgefghfghighij -> Match: True


## Problem 5: Passwords

In [46]:
# Target password hashes
target_hashes = [
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342"
]

In [47]:
# Dictionary attack using common passwords
common_passwords = [
    "password", "123456", "12345678", "qwerty", "abc123", "monkey",
    "letmein", "trustno1", "dragon", "baseball", "iloveyou",
    "P@ssw0rd", "Password1", "admin", "welcome", "cheese"
]

print("Attempting dictionary attack...\n")
found = {}

for password in common_passwords:
    h = sha256(password.encode('utf-8'))
    for i, target in enumerate(target_hashes):
        if h == target:
            found[i] = password
            print(f"Found password {i+1}: '{password}'")

print(f"\nCracked {len(found)} out of 3 passwords")

Attempting dictionary attack...

Found password 1: 'password'
Found password 3: 'P@ssw0rd'
Found password 2: 'cheese'

Cracked 3 out of 3 passwords


In [48]:
# Verify the actual passwords (found using CrackStation)
actual_passwords = ["password", "cheese", "P@ssw0rd"]

print("Verification of actual passwords:")
for i, (password, target) in enumerate(zip(actual_passwords, target_hashes)):
    computed = sha256(password.encode('utf-8'))
    match = computed == target
    print(f"Password {i+1}: '{password}' -> Match: {match}")

Verification of actual passwords:
Password 1: 'password' -> Match: True
Password 2: 'cheese' -> Match: True
Password 3: 'P@ssw0rd' -> Match: True


### How the Passwords Were Found

Dictionary Attack Method:
1. Compiled a list of common passwords from various sources
2. Hashed each password using SHA-256
3. Compared each hash against the target hashes
4. When a match was found, the password was recovered

This works because:
1. No salt: Same password always produces the same hash
2. Single-pass SHA-256: Very fast to compute 
3. Common passwords: All three are in common password lists and breach databases

### Security Improvements

1. Salt: Add random value to each password before hashing
2. KDFs: Use bcrypt, scrypt, or Argon2 instead of plain SHA-256
3. Iterations: Hash multiple times 
4. Pepper: Add secret key to all passwords
5. Strong passwords: Longer passwords, password managers

In [50]:
# Demonstration of improved password hashing
def secure_password_hash(password, salt=None, iterations=100000):
   
    # Generate random salt if not provided
    if salt is None:
        salt = os.urandom(16)  # 128-bit salt
    
    # Combine password and salt
    salted = password.encode('utf-8') + salt
    
    # Apply multiple iterations
    current = salted
    for _ in range(iterations):
        current = sha256(current).encode('utf-8')
    
    return salt.hex(), current.decode('utf-8')

# Demonstrate that same password produces different hashes with different salts
password = "password"
salt1, hash1 = secure_password_hash(password)
salt2, hash2 = secure_password_hash(password)

print("Demonstration of salted hashing:")
print(f"Password: '{password}'")
print(f"\nWith salt 1:")
print(f"  Salt: {salt1}")
print(f"  Hash: {hash1[:32]}...")
print(f"\nWith salt 2:")
print(f"  Salt: {salt2}")
print(f"  Hash: {hash2[:32]}...")
print(f"\nHashes are different: {hash1 != hash2}")


Demonstration of salted hashing:
Password: 'password'

With salt 1:
  Salt: 78c6ec8fcb1ccb1b855870e97aff3668
  Hash: 6e44858d17f6def5c97141dda8e92c60...

With salt 2:
  Salt: 6a6b74e1137847efec408f8eddb7338b
  Hash: ba793a29f9100eb05fb8f9035df28936...

Hashes are different: True


## END